# CNN for many subjects


Primero:

Cargamos los datos y los normalizamos. Para esto, primero pasamos un filtro pasa-bajo de 0 a 20hz, luego lo normalizamos a $N(0, 1)$

In [1]:
%pylab
%matplotlib inline

import sys
if ".." not in sys.path:
    sys.path.append("..")
import glob
import os
import mne
from keras import backend as K
from p300.preprocessing import normalize_subject, load_data

print("GPU's disponibles = {}".format(K.tensorflow_backend._get_available_gpus()))

CORPORA_PATH = "~/projects/corpora/P3Speller/P3Speller-old-y-datos/sets"

file_path = os.path.expanduser(CORPORA_PATH)
files = glob.glob(os.path.join(file_path, "*.set"))



Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


/home/jmperez/.pyenv/versions/3.6.5/envs/p300/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


GPU's disponibles = ['/job:localhost/replica:0/task:0/device:GPU:0']


Targets appear as 2 in the third column


We remove last channel as well

In [2]:
%%capture

# this line is to avoid output

no_subjects_to_use = 30

training_files = files[:no_subjects_to_use]
testing_files = files[no_subjects_to_use:no_subjects_to_use+15]

X_train, y_train = load_data(training_files)
X_test, y_test = load_data(testing_files)

X_train = X_train[:, :, :, np.newaxis]
X_test = X_test[:, :, :, np.newaxis]

In [3]:
from sklearn.utils import class_weight
y_t = y_train.reshape(-1)
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_t), y_t)

class_weights = dict(zip([0,1], class_weights))

In [4]:
from keras.models import Sequential
from keras.layers import Conv1D, Conv2D, Flatten, Dense, Dropout

model = Sequential()
activation = 'relu'

n_kernels = 15
model.add(Conv2D(n_kernels, (14, 1), padding='same', 
                activation=activation, input_shape=(14, 104, 1)))
model.add(Conv2D(3*n_kernels, (1, 13), padding='same',
                activation=activation))
model.add(Flatten())
model.add(Dropout(0.45))
model.add(Dense(128, activation=activation))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', # using the cross-entropy loss function
              optimizer='rmsprop', 
              metrics=['accuracy']) # reporting the accuracy


Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [5]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpointer = ModelCheckpoint(filepath='model.h5', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

model.fit(
    X_train, y_train, epochs=20, 
    batch_size=256, class_weight=class_weights, validation_split=0.10,
    callbacks=[checkpointer]
)

Train on 52650 samples, validate on 5850 samples
Epoch 1/20
52650/52650 [==============================] - 10s 194us/step - loss: 0.7169 - acc: 0.5267 - val_loss: 0.6921 - val_acc: 0.3942
Epoch 2/20
52650/52650 [==============================] - 5s 96us/step - loss: 0.6764 - acc: 0.5801 - val_loss: 0.6920 - val_acc: 0.6879
Epoch 3/20
52650/52650 [==============================] - 5s 93us/step - loss: 0.6654 - acc: 0.6121 - val_loss: 0.6945 - val_acc: 0.4674
Epoch 4/20
52650/52650 [==============================] - 5s 94us/step - loss: 0.6551 - acc: 0.6274 - val_loss: 0.6932 - val_acc: 0.6359
Epoch 5/20
52650/52650 [==============================] - 5s 93us/step - loss: 0.6440 - acc: 0.6488 - val_loss: 0.6938 - val_acc: 0.6694
Epoch 6/20
52650/52650 [==============================] - 5s 93us/step - loss: 0.6364 - acc: 0.6586 - val_loss: 0.7195 - val_acc: 0.7395
Epoch 7/20
52650/52650 [==============================] - 5s 93us/step - loss: 0.6266 - acc: 0.6702 - val_loss: 0.7247 - val_ac

In [6]:
y_pred = model.predict_classes(X_test)
y_prob = model.predict(X_test)


from sklearn.metrics import precision_score, recall_score, roc_auc_score, accuracy_score

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_prob)
accuracy = accuracy_score(y_test, y_pred)

print("""
Accuracy   = {}
Precision  = {}
Recall     = {}
ROC AUC    = {}
""".format(accuracy, precision, recall, auc))


Accuracy   = 0.6395209580838324
Precision  = 0.2071773220747889
Recall     = 0.411377245508982
ROC AUC    = 0.5703272735965196



In [7]:
from keras.models import load_model

model_2 = load_model("model.h5")

y_pred = model_2.predict_classes(X_test)
y_prob = model_2.predict(X_test)


from sklearn.metrics import precision_score, recall_score, roc_auc_score, accuracy_score

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_prob)
accuracy = accuracy_score(y_test, y_pred)

print("""
Accuracy   = {}
Precision  = {}
Recall     = {}
ROC AUC    = {}
""".format(accuracy, precision, recall, auc))


Accuracy   = 0.685395874916833
Precision  = 0.23899518722854796
Recall     = 0.4063872255489022
ROC AUC    = 0.6088371679794105



Accuracy   = 0.5315972222222223
Precision  = 0.1865079365079365
Recall     = 0.5385416666666667
ROC AUC    = 0.5470785026041667

In [24]:
from keras.models import Sequential
from keras.layers import Conv1D, Conv2D, Flatten, Dense, Dropout, MaxPool2D

model = Sequential()

n_kernels = 10
model.add(Conv2D(n_kernels, (14, 1), padding='same', 
                activation='relu', input_shape=(14, 104, 1)))
model.add(Conv2D(5*n_kernels, (1, 13), padding='same',
                activation='relu'))
model.add(MaxPool2D((1, 4)))
model.add(Flatten())
model.add(Dropout(0.45))
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', # using the cross-entropy loss function
              optimizer='rmsprop', 
              metrics=['accuracy']) # reporting the accuracy

In [25]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpointer = ModelCheckpoint(filepath='model.with_maxpool.h5', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

model.fit(
    X_train, y_train, epochs=30, 
    batch_size=256, class_weight={0:1, 1:6}, validation_split=0.01,
    callbacks=[checkpointer]
)

Train on 87674 samples, validate on 886 samples
Epoch 1/30
87674/87674 [==============================] - 6s 74us/step - loss: 1.2840 - acc: 0.3576 - val_loss: 1.2286 - val_acc: 0.3149
Epoch 2/30
87674/87674 [==============================] - 6s 65us/step - loss: 1.2310 - acc: 0.4305 - val_loss: 1.2184 - val_acc: 0.2291
Epoch 3/30
87674/87674 [==============================] - 6s 65us/step - loss: 1.2223 - acc: 0.4579 - val_loss: 1.1952 - val_acc: 0.4571
Epoch 4/30
87674/87674 [==============================] - 6s 65us/step - loss: 1.2132 - acc: 0.4865 - val_loss: 1.1990 - val_acc: 0.4819
Epoch 5/30
87674/87674 [==============================] - 6s 65us/step - loss: 1.2044 - acc: 0.5097 - val_loss: 1.1752 - val_acc: 0.5609
Epoch 6/30
87674/87674 [==============================] - 6s 65us/step - loss: 1.1983 - acc: 0.5194 - val_loss: 1.1720 - val_acc: 0.4187
Epoch 7/30
87674/87674 [==============================] - 6s 64us/step - loss: 1.1950 - acc: 0.5168 - val_loss: 1.1759 - val_acc: 

In [26]:
from keras.models import load_model


y_pred = model.predict_classes(X_test)
y_prob = model.predict(X_test)


from sklearn.metrics import precision_score, recall_score, roc_auc_score, accuracy_score

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_prob)
accuracy = accuracy_score(y_test, y_pred)

print("""
Accuracy   = {}
Precision  = {}
Recall     = {}
ROC AUC    = {}
""".format(accuracy, precision, recall, auc))


Accuracy   = 0.6778472222222223
Precision  = 0.20906964656964658
Recall     = 0.33520833333333333
ROC AUC    = 0.5621011501736111



## Model with two layers

In [28]:
from keras.models import Sequential
from keras.layers import Conv1D, Conv2D, Flatten, Dense, Dropout, MaxPool2D

model = Sequential()

n_kernels = 10
model.add(Conv2D(2*n_kernels, (14, 1), padding='same', 
                activation='relu', input_shape=(14, 104, 1)))
model.add(Conv2D(5*n_kernels, (1, 13), padding='same',
                activation='relu'))
model.add(MaxPool2D((1, 4)))
model.add(Dropout(0.35))
model.add(Conv2D(n_kernels, (14, 1), padding='same', 
                activation='relu', input_shape=(14, 104, 1)))
model.add(Conv2D(2*n_kernels, (1, 13), padding='same',
                activation='relu'))
model.add(MaxPool2D((1, 4)))

model.add(Flatten())
model.add(Dropout(0.35))
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', # using the cross-entropy loss function
              optimizer='rmsprop', 
              metrics=['accuracy']) # reporting the accuracy

In [29]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpointer = ModelCheckpoint(filepath='model.2conv_with_maxpool.h5', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

model.fit(
    X_train, y_train, epochs=30, 
    batch_size=256, class_weight={0:1, 1:6}, validation_split=0.01,
    callbacks=[checkpointer]
)

Train on 87674 samples, validate on 886 samples
Epoch 1/30
87674/87674 [==============================] - 9s 97us/step - loss: 1.2578 - acc: 0.2909 - val_loss: 1.2336 - val_acc: 0.3059
Epoch 2/30
87674/87674 [==============================] - 7s 84us/step - loss: 1.2384 - acc: 0.4181 - val_loss: 1.2232 - val_acc: 0.3341
Epoch 3/30
87674/87674 [==============================] - 7s 84us/step - loss: 1.2309 - acc: 0.4623 - val_loss: 1.2136 - val_acc: 0.4300
Epoch 4/30
87674/87674 [==============================] - 7s 84us/step - loss: 1.2239 - acc: 0.4719 - val_loss: 1.2004 - val_acc: 0.5609
Epoch 5/30
87674/87674 [==============================] - 7s 84us/step - loss: 1.2187 - acc: 0.4972 - val_loss: 1.1912 - val_acc: 0.4447
Epoch 6/30
87674/87674 [==============================] - 7s 84us/step - loss: 1.2165 - acc: 0.4848 - val_loss: 1.1886 - val_acc: 0.4808
Epoch 7/30
87674/87674 [==============================] - 7s 84us/step - loss: 1.2145 - acc: 0.5036 - val_loss: 1.1881 - val_acc: 

In [30]:
from keras.models import load_model
from sklearn.metrics import precision_score, recall_score, roc_auc_score, accuracy_score


y_pred = model.predict_classes(X_test)
y_prob = model.predict(X_test)

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_prob)
accuracy = accuracy_score(y_test, y_pred)

print("""
Accuracy   = {}
Precision  = {}
Recall     = {}
ROC AUC    = {}
""".format(accuracy, precision, recall, auc))


Accuracy   = 0.4284375
Precision  = 0.18658280922431866
Recall     = 0.723125
ROC AUC    = 0.5781777647569444



## Conv with two layers but one simple

In [31]:
from keras.models import Sequential
from keras.layers import Conv1D, Conv2D, Flatten, Dense, Dropout, MaxPool2D

model = Sequential()

n_kernels = 10
model.add(Conv2D(2*n_kernels, (14, 1), padding='same', 
                activation='relu', input_shape=(14, 104, 1)))
model.add(Conv2D(5*n_kernels, (1, 13), padding='same',
                activation='relu'))
model.add(MaxPool2D((1, 4)))
model.add(Conv2D(n_kernels, (14, 5), padding='same', 
                activation='relu', input_shape=(14, 104, 1)))
model.add(Flatten())
model.add(Dropout(0.35))
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', # using the cross-entropy loss function
              optimizer='rmsprop', 
              metrics=['accuracy']) # reporting the accuracy

In [32]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpointer = ModelCheckpoint(filepath='model.2conv_with_maxpool.h5', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

model.fit(
    X_train, y_train, epochs=30, 
    batch_size=256, class_weight={0:1, 1:6}, validation_split=0.01,
    callbacks=[checkpointer, early_stopping]
)

Train on 87674 samples, validate on 886 samples
Epoch 1/30
87674/87674 [==============================] - 8s 88us/step - loss: 1.2603 - acc: 0.3354 - val_loss: 1.2379 - val_acc: 0.6806
Epoch 2/30
87674/87674 [==============================] - 7s 77us/step - loss: 1.2385 - acc: 0.4356 - val_loss: 1.2346 - val_acc: 0.5023
Epoch 3/30
87674/87674 [==============================] - 7s 78us/step - loss: 1.2303 - acc: 0.4524 - val_loss: 1.2322 - val_acc: 0.3296
Epoch 4/30
87674/87674 [==============================] - 7s 78us/step - loss: 1.2262 - acc: 0.4699 - val_loss: 1.2080 - val_acc: 0.4707
Epoch 5/30
87674/87674 [==============================] - 7s 78us/step - loss: 1.2189 - acc: 0.4926 - val_loss: 1.2386 - val_acc: 0.2269
Epoch 6/30
87674/87674 [==============================] - 7s 78us/step - loss: 1.2165 - acc: 0.4894 - val_loss: 1.2285 - val_acc: 0.4289
Epoch 7/30
87674/87674 [==============================] - 7s 78us/step - loss: 1.2120 - acc: 0.5042 - val_loss: 1.2172 - val_acc: 

In [33]:
from keras.models import load_model


y_pred = model.predict_classes(X_test)
y_prob = model.predict(X_test)


from sklearn.metrics import precision_score, recall_score, roc_auc_score, accuracy_score

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_prob)
accuracy = accuracy_score(y_test, y_pred)

print("""
Accuracy   = {}
Precision  = {}
Recall     = {}
ROC AUC    = {}
""".format(accuracy, precision, recall, auc))


Accuracy   = 0.48006944444444444
Precision  = 0.19064704451471662
Recall     = 0.653125
ROC AUC    = 0.5707269444444445

